In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
from animal_shelter import AnimalShelter

username = "aacuser"
password = "aacuserpw"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
df = pd.DataFrame.from_records(shelter.readAll({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')


image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    #client logo in upper left corner of screen, URL anchor tag on logo to www.snhu.edu 
    html.A([
        html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image',
                style={
                    'height' : '7%',
                    'width' : '7%',
                    'float' : 'left',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                }),
    ], href='https://www.snhu.edu'),

    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('By Sarah Deleppo'))), #identifier
    html.Hr(),
    
    # Radio items listed above data table and charts
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          dcc.RadioItems(
                              id='radio',
                            options=[
                            {'label': 'Water Rescue', 'value': 'water'},
                            {'label': 'Mountain Rescue', 'value': 'mt'},
                            {'label': 'Disaster Rescue', 'value': 'disaster'},
                            {'label': 'Reset Data', 'value': 'all'}    
                            ],
                            value='MTL'
                            )
                  ]),
    #interactive data table
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #display charts side by side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
    html.Div(
            dcc.Graph(id="pie-chart"),
            className='col s12 m6',
            ),
    html.Div(
            id='map-id',
            className='col s12 m6',
            )])
        

])

#############################################
# Interaction Between Components / Controller
#############################################

#radio item filters with queries for client specifications
@app.callback(Output('datatable-interactivity',"data"), 
              [Input('radio', 'value')])
def update_table(value):
    if (value == 'water'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {'$in':["Labrador Retriever Mix", 
                                                                "Chesapeake Bay Retriever",
                                                                 "Newfoundland"]},
                                                "sex_upon_outcome": "Intact Female",
                                               "age_upon_outcome_in_weeks" : {'$gt': 26, '$lt' : 156}})))
    elif (value == 'mt'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {'$in':["German Shepherd", 
                                                                "Alaskan Malamute",
                                                                 "Old English Sheepdog",
                                                                 "Siberian Husky",
                                                                 "Rottweiler"]},
                                                "sex_upon_outcome": "Intact Male",
                                               "age_upon_outcome_in_weeks" : {'$gt': 26, '$lt' : 156}})))
        
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {'$in':["Doberman Pinscher", 
                                                                "German Shepherd",
                                                                 "Golden Retriever",
                                                                 "Bloodhound",
                                                                 "Rottweiler"]},
                                                "sex_upon_outcome": "Intact Male",
                                               "age_upon_outcome_in_weeks" : {'$gt': 20, '$lt' : 300}})))
    elif (value == 'all'):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    
    return df.to_dict('records')

#geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

#pie chart showing breeds 
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-interactivity", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)

    fig = px.pie(
        dff,
        names='breed',
    )

    return fig


app
